# Objectif: extraire les données utiles des fichier GRIB en gardant les informations de géolocalisation



In [1]:
%pip install "dask[diagnostics]" tqdm

Looking in indexes: https://pypi.org/simple, https://gitlab.ecoco2.com/api/v4/groups/recherche/-/packages/pypi/simple

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import xarray as xr
from pathlib import Path
from tqdm.auto import tqdm

/home/ubuntu/.local/share/hatch/env/virtual/energy-forecast/S-pRwtAe/energy-forecast/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
start_date = "2022-02-01"
end_date = "2024-04-08"
list_date = pd.date_range(start=start_date, end=end_date, freq="D")
list_date = list_date.strftime("%Y-%m-%d")
cases_forecast = {
    "J+0": ["00H12H", "13H24H"],
    "J+1": ["25H36H", "37H48H"],
    "J+2": ["49H60H", "61H72H"],
    "J+3": ["73H84H", "85H96H"],
}

all_cases = [
    "00H12H",
    "13H24H",
    "25H36H",
    "37H48H",
    "49H60H",
    "61H72H",
    "73H84H",
    "85H96H",
]

filename_save_template = (
    "/shared/home/antoine-2etavant/data/arpege/{date}_{SPVALUE}_{case}.grib2"
)

new_filename_save_template = (
    "/shared/home/antoine-2etavant/data/weather/{variable}/{date}.nc"
)

In [4]:
KEYS_FILTER_SSPD = {
    "typeOfLevel": "surface",
    "cfVarName": "ssrd",
}
KEYS_FILTER_WIND = {
    "typeOfLevel": "heightAboveGround",
    "level": 10,
    "cfVarName": "si10",
}
KEYS_FILTER_T2M = {
    "typeOfLevel": "heightAboveGround",
    "level": 2,
    "cfVarName": "t2m",
}
KEYS_FILTER_SSPD_TOTAL = {
    "typeOfLevel": "surface",
    "cfVarName": "ssr",
}

In [5]:
import yaml

filename_france_bounds = "../../data/geo/france_bounds.yml"

with open(filename_france_bounds, "r") as stream:
    france_bounds = yaml.safe_load(stream)
france_bounds

{'min_lon': -4.79542,
 'max_lon': 9.55996,
 'min_lat': 41.36484,
 'max_lat': 51.089}

In [6]:
def select_region(ds, min_lon, max_lon, min_lat, max_lat):
    """Check if 'step' is in the dataset's coordinates
    and return the dataset with the selected region.
    """
    if "step" not in ds.coords:
        print("step not in coords")
        print(ds.encoding["source"])
        return None
    if "longitude" not in ds.coords:
        print("longitude not in coords")
        return None
    if "latitude" not in ds.coords:
        print("latitude not in coords")
        return None
    return ds.sel(longitude=slice(min_lon, max_lon), latitude=slice(max_lat, min_lat))


def select_france(ds):
    return select_region(ds, **france_bounds)

In [7]:
date = list_date[0]


def process_one_date_generic(
    date: str,
    variable_name: str,
    force_recompute: bool = False,
    open_file_callback=None,
    spvalue="SP1",
):
    new_filename = Path(
        new_filename_save_template.format(date=date, variable=variable_name)
    )
    if new_filename.exists() and not force_recompute:
        print(f"File {new_filename} already exists")
        return
    list_filenames = [
        filename_save_template.format(date=date, case=case, SPVALUE=spvalue)
        for case in all_cases
    ]
    list_of_filenames_from_case_exists = [
        filename for filename in list_filenames if Path(filename).exists()
    ]
    if len(list_of_filenames_from_case_exists) == 0:
        print(f"No file found for date {date}")
        return
    da = open_file_callback(list_of_filenames_from_case_exists)
    new_filename.parent.mkdir(parents=True, exist_ok=True)
    try:
        da.to_netcdf(new_filename)
    except Exception as e:
        print(f"Error while saving {new_filename}: {e}")
    return


def open_windspeed(list_of_filenames_from_case_exists):
    da_wind_10m = xr.concat(
        [
            xr.open_mfdataset(
                filenames,
                engine="cfgrib",
                # parallel=False,
                backend_kwargs={"filter_by_keys": KEYS_FILTER_WIND},
                concat_dim="time",
                preprocess=select_france,
                combine="nested",
                # chunks={"time": 10},
            ).si10.drop_vars("heightAboveGround")
            for filenames in list_of_filenames_from_case_exists
        ],
        dim="step",
    )
    da_wind_10m.name = "wind_speed"
    return da_wind_10m


def process_one_date_wind(date, force_recompute=False):
    process_one_date_generic(
        date,
        "wind_speed",
        spvalue="SP1",
        force_recompute=force_recompute,
        open_file_callback=open_windspeed,
    )


def open_sunflux(list_of_filenames_from_case_exists):
    da_sunflux = xr.concat(
        [
            xr.open_mfdataset(
                filenames,
                engine="cfgrib",
                # parallel=False,
                backend_kwargs={"filter_by_keys": KEYS_FILTER_SSPD},
                concat_dim="time",
                preprocess=select_france,
                combine="nested",
                # chunks={"time": 10},
            ).ssrd.drop_vars("surface")
            for filenames in list_of_filenames_from_case_exists
        ],
        dim="step",
    )
    da_sunflux.name = "sun_flux"
    return da_sunflux


def open_temperature(list_of_filenames_from_case_exists):
    da_temperature = xr.concat(
        [
            xr.open_mfdataset(
                filenames,
                engine="cfgrib",
                # parallel=False,
                backend_kwargs={"filter_by_keys": KEYS_FILTER_T2M},
                concat_dim="time",
                preprocess=select_france,
                combine="nested",
                # chunks={"time": 10},
            ).t2m.drop_vars("heightAboveGround")
            for filenames in list_of_filenames_from_case_exists
        ],
        dim="step",
    )
    da_temperature.name = "temperature"
    return da_temperature


def process_one_date_sunflux(date, force_recompute=False):
    process_one_date_generic(
        date,
        "sun_flux",
        spvalue="SP1",
        force_recompute=force_recompute,
        open_file_callback=open_sunflux,
    )


def process_one_date_temperature(date, force_recompute=False):
    process_one_date_generic(
        date,
        "temperature",
        spvalue="SP1",
        force_recompute=force_recompute,
        open_file_callback=open_temperature,
    )

In [8]:
def open_sunflux_total(list_of_filenames_from_case_exists):
    da_sunflux = xr.concat(
        [
            xr.open_mfdataset(
                filename,
                engine="cfgrib",
                # parallel=False,
                backend_kwargs={"filter_by_keys": KEYS_FILTER_SSPD_TOTAL},
                concat_dim="time",
                preprocess=select_france,
                combine="nested",
                # chunks={"time": 10},
            ).ssr.drop_vars("surface")
            for filename in list_of_filenames_from_case_exists
        ],
        dim="step",
    )
    da_sunflux.name = "sun_flux_total"
    return da_sunflux


def process_one_date_sunfluxtotal(date, force_recompute=False):
    process_one_date_generic(
        date,
        "sun_flux_total",
        spvalue="SP2",
        force_recompute=force_recompute,
        open_file_callback=open_sunflux_total,
    )

In [9]:
process_one_date_sunflux(date, force_recompute=True)
process_one_date_temperature(date, force_recompute=True)
process_one_date_wind(date, force_recompute=True)
process_one_date_sunfluxtotal(date, force_recompute=True)

In [10]:
process_one_date_sunflux(date, force_recompute=False)
process_one_date_temperature(date, force_recompute=False)
process_one_date_wind(date, force_recompute=False)
process_one_date_sunfluxtotal(date, force_recompute=False)

File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux_total/2022-02-01.nc already exists


In [11]:
def process_one_date(date):
    process_one_date_sunflux(date)
    process_one_date_temperature(date)
    process_one_date_wind(date)
    process_one_date_sunfluxtotal(date)


def process_all_dates(list_date) -> None:
    for date in tqdm(list_date, maxinterval=1, mininterval=0.1):
        process_one_date(date)

In [12]:
process_all_dates(list_date)

  4%|▎         | 28/798 [00:00<00:02, 267.65it/s]

File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux_total/2022-02-01.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-02-02.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-02-02.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-02-02.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux_total/2022-02-02.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-02-03.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-02-03.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-02-03.nc already exists
File /shared/home/antoine-2

Can't read index file '/shared/home/antoine-2etavant/data/arpege/2022-03-27_SP2_61H72H.grib2.789e2.idx'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/energy-forecast/S-pRwtAe/energy-forecast/lib/python3.10/site-packages/cfgrib/messages.py", line 547, in from_indexpath_or_filestream
    self = cls.from_indexpath(indexpath)
  File "/home/ubuntu/.local/share/hatch/env/virtual/energy-forecast/S-pRwtAe/energy-forecast/lib/python3.10/site-packages/cfgrib/messages.py", line 429, in from_indexpath
    index = pickle.load(file)
EOFError: Ran out of input


File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-03-25.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-03-25.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-03-25.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux_total/2022-03-25.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-03-26.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-03-26.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-03-26.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux_total/2022-03-26.nc already exists
File /shared/home/antoine-2etavant/data/weather/sun_flux/2022-03-27.nc already exists
File /shared/home/antoine-2etavant/data/weather/temperature/2022-03-27.nc already exists
File /shared/home/antoine-2etavant/data/weather/wind_speed/2022-03-27.nc already exists


  7%|▋         | 55/798 [00:14<03:54,  3.16it/s] 

...

 98%|█████████▊| 782/798 [18:02:02<29:21, 110.12s/it]

No file found for date 2024-03-24
No file found for date 2024-03-24
No file found for date 2024-03-24
No file found for date 2024-03-24


 99%|█████████▉| 789/798 [18:13:05<15:44, 104.97s/it]

No file found for date 2024-03-31
No file found for date 2024-03-31
No file found for date 2024-03-31
No file found for date 2024-03-31


100%|██████████| 798/798 [18:27:30<00:00, 83.27s/it] 


# Ouvrir les fichier et autres verifications

In [2]:
import xarray as xr
from pathlib import Path
import pandas as pd

In [3]:
root_path_ssrd = Path("/shared/home/antoine-2etavant/data/weather/sun_flux/")
root_path_si10 = Path("/shared/home/antoine-2etavant/data/weather/wind_speed/")
root_path_t2m = Path("/shared/home/antoine-2etavant/data/weather/temperature/")
root_path_ssr = Path("/shared/home/antoine-2etavant/data/weather/sun_flux_total/")

In [4]:
files_ssrd = list(root_path_ssrd.glob("*.nc"))
n_files_ssrd = len(files_ssrd)
files_si10 = list(root_path_si10.glob("*.nc"))
n_files_si10 = len(files_si10)
files_t2m = list(root_path_t2m.glob("*.nc"))
n_files_t2m = len(files_t2m)
files_ssr = list(root_path_ssr.glob("*.nc"))
n_files_ssr = len(files_ssr)


print(f"Number of files for ssrd: {n_files_ssrd}")
print(f"Number of files for si10: {n_files_si10}")
print(f"Number of files for t2m: {n_files_t2m}")
print(f"Number of files for ssr: {n_files_ssr}")

Number of files for ssrd: 772
Number of files for si10: 772
Number of files for t2m: 772
Number of files for ssr: 772


In [5]:
print("processing ssr")
ds_ssr = xr.open_mfdataset(files_ssr, combine="by_coords")
ds_ssr.to_netcdf("/shared/home/antoine-2etavant/data/weather/sun_flux_total.nc")
print("processing ssrd")
ds_ssrd = xr.open_mfdataset(files_ssrd, combine="by_coords")
ds_ssrd.to_netcdf("/shared/home/antoine-2etavant/data/weather/sun_flux_downward.nc")
print("processing si10")
ds_si10 = xr.open_mfdataset(files_si10, combine="by_coords")
ds_si10.to_netcdf("/shared/home/antoine-2etavant/data/weather/wind_speed.nc")
print("processing t2m")
ds_t2m = xr.open_mfdataset(files_t2m, combine="by_coords")
ds_t2m.to_netcdf("/shared/home/antoine-2etavant/data/weather/temperature.nc")
print("done")

processing ssr
processing ssrd
processing si10
processing t2m
done
